In [ ]:
#ライブラリのインポート欄
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest
from sklearn import model_selection
from sklearn.model_selection import GridSearchCV
import warnings
import seaborn as sns #可視化
from sklearn.preprocessing import StandardScaler

warnings.filterwarnings('ignore')

In [ ]:
import zipfile#サンプルがzipなので展開する
zipfile.ZipFile('/kaggle/input/ghouls-goblins-and-ghosts-boo/train.csv.zip').extractall() 
zipfile.ZipFile('/kaggle/input/ghouls-goblins-and-ghosts-boo/test.csv.zip').extractall()
#zipfile.ZipFile('/kaggle/input/ghouls-goblins-and-ghosts-boo/sample_submission.csv.zip').extractall()

In [ ]:
# CSVを読み込む

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
# 提出用に（分ける）
# データの統合（分けて適用するのが面倒なため）

In [ ]:
# csv内のヘッダーを確認
import csv
csv_file = open("train.csv", "r", encoding="ms932", errors="", newline="" )
f = csv.reader(csv_file, delimiter=",", doublequote=True, lineterminator="\r\n", quotechar='"', skipinitialspace=True)
header = next(f)
print(header)

In [ ]:
# 骨の長さの分布
sns.stripplot(x='type',y='bone_length',data=train)

In [ ]:
# 腐敗度の分布
sns.stripplot(x='type',y='rotting_flesh',data=train)

In [ ]:
# 髪の長さの分布
sns.stripplot(x='type',y='hair_length',data=train)

In [ ]:
# 魂の割合の分布
sns.stripplot(x='type',y='has_soul',data=train)

In [ ]:
# 色ごとの分布図
sns.countplot(x = 'color', hue = "type", data = train)

In [ ]:
## 主成分分析
pd.get_dummies(train['color'])
pd.get_dummies(test['color'])

training = pd.get_dummies(train, columns=['color'])

# id と type を取り除く
df_training = training.iloc[:, 1:]
left = df_training.iloc[:, :4]
# right = df_training.iloc[:, 5:]
# df_training = left.join(right)
df_training = df_training.iloc[:, :4]

# 標準化
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
trainings = ss.fit_transform(df_training)
trainings = pd.DataFrame(trainings, columns=df_training.columns[:])
trainings.head()

# 主成分変換
from sklearn.decomposition import PCA
pca = PCA()
pca.fit(trainings) #データを学習
feat = pca.transform(trainings) #データを成形
feat = pd.DataFrame(feat, columns = ['pc{}'.format(x+1) for x in range(len(trainings.columns))])
feat.head()

# typeを数値化
training_c = pd.get_dummies(train, columns=['color'])
training_c['type'] = training_c['type'].astype('category').cat.codes
training_c.head()

# A = np.append(np.zeros(350),np.ones(11))
# 図
import matplotlib.pyplot as plt
plt.figure(figsize = (6,6))
plt.scatter(feat['pc1'], feat['pc2'], alpha = 0.8,c = training_c['type'])
# plt.scatter(feat['pc1'], feat['pc2'], alpha = 0.8, c=np.append(A,np.full(10,2))
plt.grid()
plt.xlabel('pc1')
plt.ylabel('pc2')
# plt.legend('c')
plt.show()
print('紫：Ghost','緑：Ghoul','黄：Goblin')

In [ ]:
# 相関のヒートマップ
cor = training[[ 'bone_length', 'rotting_flesh', 'hair_length', 'has_soul','color_black','color_blood','color_blue','color_clear','color_green','color_white']].corr()
sns.heatmap(cor, cmap= sns.color_palette('coolwarm', 10), annot=True,fmt='.2f', vmin = -1, vmax = 1)
plt.savefig('seaborn_heatmap_corr_example.png')
print(cor)

In [ ]:
#このプログラムは上から実行するので、再実行は上のプログラムからやり直さないとダメ
#予想を表から除いてダミー化
train2 = pd.get_dummies(train['type'])
del train['type']
COLOR = pd.get_dummies(train['color'])
del train['color']
del train['id']
#ダミー化した結果をクラスとして格納
target = pd.DataFrame(train2['Ghost']* 0 + train2['Ghoul'] * 1 + train2['Goblin'] * 2)
target_GOB = pd.DataFrame(train2['Ghost']* 0 + train2['Ghoul'] * 0 + train2['Goblin'] * 1)
target = target.rename(columns={'0': 'target'})
#予想を表から除いてダミー化(テストデータも)
COLOR2 = pd.get_dummies(test['color'])
del test['color']
ID = test["id"]
del test['id']

In [ ]:
import numpy as np
%matplotlib inline
from sklearn import datasets
from sklearn.model_selection import train_test_split # クロスバリデーション用（テストとトレ分ける）
from sklearn.model_selection import cross_val_score
from sklearn import metrics       # 精度検証用
from sklearn.metrics import accuracy_score

# ニューラルネットワーク
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn import metrics
import joblib

In [ ]:
#ゴブリンだけ色を入れて予想
train_c = train.join(COLOR)
test_c = test.join(COLOR2)
#target_c = target.join(COLOR)

In [ ]:
def GOB():
    global y_pred_c
    global train
    global test
    # paramate
    hidden_layer_sizes=(20,30)
    activation = 'relu'
    solver = 'adam'
    batch_size = 'auto'
    alpha = 0.0001
    random_state = 0
    max_iter = 10000
    early_stopping = True

    # 説明変数

    train_X = train
    # 目的変数

    train_y1 = target_GOB

    # 学習
    clf = MLPRegressor(
        hidden_layer_sizes=hidden_layer_sizes,
        activation=activation,
        solver=solver,
        batch_size=batch_size,
        alpha=alpha,
        random_state=random_state,
        max_iter=max_iter,
    #     early_stopping = early_stopping
        )

    clf.fit(train_X, train_y1)

    SAVE_TRAINED_DATA_PATH = 'train1.learn'
    # 学習結果を出力
    joblib.dump(clf, SAVE_TRAINED_DATA_PATH)

    # 学習済ファイルのロード
    clf1 = joblib.load(SAVE_TRAINED_DATA_PATH)

    # 検証用の説明変数

    test_X1 = test

    # 学習結果の検証
    #predict_y1 = clf1.predict_proba(train_X)
    predict_Y1 = clf1.predict(train_X)
    print(predict_Y1)
    
    y_pred_c = pd.DataFrame(predict_Y1)
    train = train.join(y_pred_c)
    
    #predict_y1 = clf1.predict_proba(test_X1)
    predict_Y1 = clf1.predict(test_X1)
    print(predict_Y1)
    y_pred_c = pd.DataFrame(predict_Y1)
    test = test.join(y_pred_c)

In [ ]:
dataset = pd.concat([train, test], ignore_index = True)
#print('isnull\n',train.isnull().sum())#欠損値がないことを確認

In [ ]:
 def lightGBM_k():
    global y_pred_AA
    # paramate
    hidden_layer_sizes=(20,30)
    activation = 'relu'
    solver = 'adam'
    batch_size = 'auto'
    alpha = 0.0001
    random_state = 0
    max_iter = 10000
    early_stopping = True

    # 説明変数

    train_X = train_k
    # 目的変数

    train_y1 = target_k

    # 学習
    clf = MLPClassifier(
        hidden_layer_sizes=hidden_layer_sizes,
        activation=activation,
        solver=solver,
        batch_size=batch_size,
        alpha=alpha,
        random_state=random_state,
        max_iter=max_iter,
    #     early_stopping = early_stopping
        )

    clf.fit(train_X, train_y1)

    SAVE_TRAINED_DATA_PATH = 'train1.learn'
    # 学習結果を出力
    joblib.dump(clf, SAVE_TRAINED_DATA_PATH)

    # 学習済ファイルのロード
    clf1 = joblib.load(SAVE_TRAINED_DATA_PATH)

    # 検証用の説明変数

    test_X1 = test_k

    # 学習結果の検証
    predict_y1 = clf1.predict_proba(test_X1)
    predict_Y1 = clf1.predict(test_X1)
    print(predict_Y1)
    y_pred_AA = predict_Y1
    # 学習データの精度
    print("学習データの精度: {:.3f}".format(clf.score(train_X, train_y1)))

    # テスト結果の精度
    print("テスト結果の精度: {:.3f}".format(clf.score(test_X1, predict_Y1)))

In [ ]:
def syuunou():
    global s_c
    global y_pred_AA
    y_pred_z:str=[]
    for n in range(len(y_pred_AA)):
        if y_pred_AA[n]==0:
            y_pred_z.append('Ghost')
        if y_pred_AA[n]==1:
            y_pred_z.append('Ghoul')
        if y_pred_AA[n]==2:
            y_pred_z.append('Goblin')
            
    
    
    s_c= pd.DataFrame({"id": ID_black, "type": y_pred_z})

In [ ]:
def color_select():
    global train
    global test
    global target
    global train_k
    global test_k
    global target_k
    global lightGBM_k
    global syuunou
    global ID_black
    global y_pred_AA
    #train_k = train[train[color] == 1]
    #test_k = test[test[color] == 1]
    #target_k = target[target[color] == 1]
    train_k = train
    test_k = test
    target_k = target
    ID_black = ID
    lightGBM_k()
    syuunou()

In [ ]:
#各色で選択する
GOB()
color_select()
sc = s_c


#並べ替え　初期設定
rows = sc.sort_values('id')




In [ ]:
##ここから推理します


# Kaggle提出用csvファイルの作成







#submission = pd.DataFrame({"id": ID, "type": y_preda})
submission = rows
submission.to_csv("submission6.csv", index=False)